In [21]:
%load_ext autoreload
%autoreload 2
import torch
from cosypose.config import LOCAL_DATA_DIR
from cosypose.datasets.datasets_cfg import make_scene_dataset
from cosypose.rendering.bullet_scene_renderer import BulletSceneRenderer
from cosypose.visualization.singleview import make_singleview_prediction_plots, filter_predictions
from cosypose.visualization.singleview import filter_predictions
from bokeh.plotting import gridplot
from bokeh.io import show, output_notebook; output_notebook()
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [22]:
result_id = 'tless-siso-n_views=1--1159351504'
ds_name, urdf_ds_name = 'tless.primesense.test', 'tless.cad'
pred_key = 'pix2pose_detections/refiner/iteration=4'

results = LOCAL_DATA_DIR / 'results' / result_id / 'results.pth.tar'
scene_ds = make_scene_dataset(ds_name)
results = torch.load(results)['predictions']

2:53:52.101393 - Building index and loading annotations...
2:42:03.917552 - Building index and loading annotations...
1:03:42.980862 - Building index and loading annotations...
0:01:09.918422 - Building index and loading annotations...


[Memory]10432.4s, 173.9min: Loading build_index...


In [23]:
results[pred_key].infos.loc[:, ['scene_id', 'view_id']].groupby('scene_id').first()

,view_id
scene_id,
1,303


In [24]:
scene_id, view_id = 1, 303

# Replace this here, you can use the dataframe above to get examples of scene/view ids.
# scene_id, view_id = 48, 733

this_preds = filter_predictions(results[pred_key], scene_id, view_id)
# obj_id = 185
# this_preds.poses = torch.unsqueeze(this_preds.poses[obj_id], 0)
# this_preds.poses_input = torch.unsqueeze(this_preds.poses_input[obj_id], 0)
# this_preds.K_crop = torch.unsqueeze(this_preds.K_crop[obj_id], 0)
# this_preds.boxes_rend = torch.unsqueeze(this_preds.boxes_rend[obj_id], 0)
# this_preds.boxes_crop = torch.unsqueeze(this_preds.boxes_crop[obj_id], 0)
# this_preds.infos = this_preds.infos[this_preds.infos.index == obj_id]
# print(this_preds.poses)
renderer = BulletSceneRenderer(urdf_ds_name)
figures = make_singleview_prediction_plots(scene_ds, renderer, this_preds)
renderer.disconnect()
print(this_preds.poses)

show(figures['input_im'])
show(figures['pred_overlay'])

KeyError: (1, 303)